<a href="https://www.kaggle.com/code/addarm/bond-valuations?scriptVersionId=189622704" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Bond Valuations Explained
<a href="https://www.kaggle.com/code/addarm/xxxt" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/adamd1985/xxx" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Bonds 2024.07.24](https://raw.githubusercontent.com/adamd1985/quant_research/main/images/bondbanner.png)

<!-- @import "[TOC]" {cmd="toc" depthFrom=1 depthTo=6 orderedList=false} -->


Bond valuation and selection is a good exercise in rounding your portfolio, mainly determining the fair value of a bond based on its future cash flows and time value of money. 
This article explores the fundamental principles of bond valuation and the understanding of their intrinsic values through calculating the present value of future cash flows, and the annual rate of returns. 

After understanding these concepts, we ought to be able to make informed decisions about our bond investments and portfolio construction.

## Notebook Setup

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

INSTALL_DEPS = False
if INSTALL_DEPS:
    # If Kaggle or Colab, you have to manage these. If local, install all
    %pip install numpy==1.23.4
    %pip install pandas==2.2.0

    IN_KAGGLE = IN_COLAB = False
try:
    # https://www.tensorflow.org/install/pip#windows-wsl2
    import google.colab
    from google.colab import drive

    drive.mount("/content/drive")
    DATA_PATH = "/content/drive/MyDrive/fixedincome_dataset"
    IN_COLAB = True
    print("Colab!")
except:
    IN_COLAB = False
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ and not IN_COLAB:
    print("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/fixedincome_dataset"
    IN_KAGGLE = True
    print("Kaggle!")
elif not IN_COLAB:
    IN_KAGGLE = False
    DATA_PATH = "./data/fixed"
    print("running localhost!")

# Treasuries and the Composition of a T-Bill

US Treasuries (T-bill, T-Notes, T-Bonds) are the safest fixed income instrument available on the market, for one reason, the US cannot default and it can increase taxes to pay off its debts.

The 3 month T-Bill is the benchmark for the risk free rate across all portfolios, at the time of writing it's between 4.5% to 5.2%. A T-bill is a 0 coupon fixed income instrument that does not pay any interest, and is just sold at a discount to its face value.

T-Bills are simple, T-Notes are closer to what you would get if you get any type of bond from any issuer, with an interest paid at a certian period. 

From our dataset, let's look at the attributes of a T-Note:

In [ ]:
treasuries_df = pd.read_csv(f'{DATA_PATH}/treas1.csv')
treasuries_df.dropna(axis=1, how='all', inplace=True)
treasuries_df.head(3)

## Present Value of Money

To understand the interinsic value of this T-Bill, we need to calculate its Present Value (PV) of money. The PV refers to the current worth of a sum of money that is to be received or paid out in the future, discounted at a specific rate to reflect the time value of money.

$$ 
\text{PV} = \frac{\text{FV}}{(1 + r)^n} 
$$

Where:
- $PV$ = Present Value
- $FV$ = Future Value (usually $100 face value)
- $r$ = Discount rate or interest rate per period
- $n$ = Number of periods

Discounting is the process of determining the present value of a future amount of money, considering a certain interest rate or discount rate. Its opposite is compounding, where you would use to find a bond's future value by rewriting the above equation:

$$
\text{FV} = \text{PV} \times (1+R)%\^N
$$

In [ ]:
FV = 100 # its the face value
r = 4.165 / 100 / 2
n = (2029 - 2023) * 2

PV = FV / (1 + r)**n
PV # Shoud be the price

There is a bit of discrepency between the price (or the Ask) of the T-Bill and the PV. To properly evaluate the bond we have other formulas.

# Bond Valuations

The value of a bond is calculated with the equation:

$$ 
\text{Bond Value} = \sum_{t=1}^{T} \frac{C_t}{(1 + r_{\text{adjusted}, t})^t} + \frac{F}{(1 + r_{\text{adjusted}, T})^T} 
$$

The coupon discounted cash flows (DCF) equation: $\sum_{t=1}^{T} \frac{C_t}{(1 + r_{\text{adjusted}, t})^t}$, can be rewritten as for simplicity:

$$
\text{DCF} = \text{C} \times (\frac{1-\frac{1}{(1+r)^N}}{r})
$$

Where:
- $ C_t $ = Cash flow (coupon payment) at time $ t $
- $ r_{\text{adjusted}, t} $ = Adjusted discount rate at time $ t $, considering taxes and inflation
- $ F $ = Face value of the bond (principal)
- $ T $ = Total number of periods (maturity)

The rate is usually the yield to maturity (YTM), or any other adjusted discount rate. 

Most financial calculators can breeze through the above formulas. We will do some python code instead to visualize this:


In [ ]:
C = 3.875 / 2  #  Semi-annual coupon payment
F = 100
r = 4.165 / 100 / 2  # Semi-annual YTM or stated discount rate
N = (2029 - 2023) * 2

DCF = C * ((1 - (1 / (1 + r)**N)) / r)
PV_face = F / (1 + r)**N

PV = DCF + PV_face
PV

98.47 is closer to the Ask's 98.60. Obviously it's a market, therefore things are not always fairly priced. The YTM is what affects the price most, in the case of this T-Bill it is stated as 4.165% (the Ask Yield).

### Yield to Maturity (YTM)

The YTM is the discounting rate assumed for the bond. Using the Bond pricing equation above, we replace the discount rate with the YTM:

$$
\text{P} = \text{C} \times (\frac{1-\frac{1}{(1+YTM)^N}}{YTM}) + \frac{F}{(1 + YTM)^N}
$$

The bond valuation formlua is a nonlinear equation (has an exponent $N$) with no closed form (complex, YTM is both numerator and denominator with exponents). If we need to find the YTM ourselves, we need to approximate it. Again, financial calculators can do this fast, though for illustration purposes we'll use a Newton-Raphson form.

Given the T-Note's information:

- Coupon Rate (C): 3.875%
- Face Value (F): 100 
- Maturity Date: December 31, 2029
- Settlement Date: January 3, 2023
- Price (P): 98.60157
- Payment Frequency: Semi-Annual
- Number of Payments (N): $2 \times 6 = 12$, 6yrs of semi-annual payments.

We want to solve the equation:

$$
P = \text{C} \times \left(\frac{1 - \frac{1}{(1 + YTM/2)^{12}}}{YTM/2}\right) + \frac{F}{(1 + YTM/2)^{12}}
$$

Where:

- $P$ = 98.60157
- $\text{C}$ = 3.875/2 = 1.9375
- $F$ = 100
- $N$ = 12

Using the Newton-Raphson method with the Initial guess of $YTM_0 = 0.04$ (4% devided by 2 for the semiannual compounding), we try to find the root of the function by its derivative:

$$
YTM_{i+1} = YTM_i - \frac{f(YTM_i)}{f'(YTM_i)}
$$

$$
f(YTM) = 98.60157 - 1.9375 \times \left(\frac{1 - \frac{1}{(1 + YTM/2)^{12}}}{YTM/2}\right) - \frac{100}{(1 + YTM/2)^{12}}
$$

$$
f'(YTM) = - \left( \frac{\partial}{\partial YTM} \left(1.9375 \times \frac{1 - \frac{1}{(1 + YTM/2)^{12}}}{YTM/2} \right) + \frac{\partial}{\partial YTM} \left( \frac{100}{(1 + YTM/2)^{12}} \right) \right)
$$

This is iterated until convergence ($|YTM_{i+1} - YTM_i|$ is sufficiently small):

Iteration 1:
- $YTM_0 = 0.04$
- Compute $f(YTM_0)$ and $f'(YTM_0)$
- Update $YTM_1$

Iteration 2:
- $YTM_1$
- Compute $f(YTM_1)$ and $f'(YTM_1)$
- Update $YTM_2$

The python below will illustrate this better:

In [ ]:
P = 98.60157
C = 3.875 / 2
F = 100
N = 12

def price_function(ytm):
    return P - (C * ((1 - 1 / (1 + ytm / 2)**N) / (ytm / 2)) + F / (1 + ytm / 2)**N)

def function_derivative(ytm):
    term1 = (C / 2) * (N / (1 + ytm / 2)**(N + 1))
    term2 = C * ((1 - 1 / (1 + ytm / 2)**N) / (ytm / 2)**2)
    term3 = (F * N) / (2 * (1 + ytm / 2)**(N + 1))
    return - (term1 - term2 + term3)

def newton_raphson_iter(initial_guess, tolerance=1e-5, max_iterations=20):
    ytm = initial_guess
    for _ in range(max_iterations):
        next_ytm = ytm - price_function(ytm) / function_derivative(ytm)
        if np.abs(next_ytm - ytm) < tolerance:
            return next_ytm
        ytm = next_ytm
    return ytm

ytm_0 = 0.04
stated_ytm = 0.04165
ytm_calculated = newton_raphson_iter(ytm_0)
print(f"Calculated: {ytm_calculated} vs stated: {stated_ytm}")

There are 2 other attributes that are looked at by sophisticated bond investors, these are the Bond's Duration and the Bond's Convexity.

Duration is the bond's price sensitivity to changes in the interest rates. This means that the price decreases as the interest rates increase, because higher yielding instruments will be issued.

Convexity is the bond's price sensitivity to it's yield. Yields go up as price goes down, if the interest rates are now higher than the bond's coupon, sellers need to up the yield by decreasing the bond's price.

These concepts serve to insulate one's portfolio from interest rates risk. In our case, we are only shopping for bonds we would like to hold to maturity, we'll accept the rates risk. 

There is one risk we won't accept though, that is default risk - we said treasuries are risk free, but other bonds are not, especially ones issued by the private sector. When shopping for bonds, select investment grade ratings from Moody's or S&P (anything in the As ratings, with AAA being the highest). 

Bonds with high risk of default have high yields to offset this, though they are not called 'junk bonds' for nothing (read about Argentina's sovereign bonds in 2001). Another risk we have to avoid is recall risk, some bonds can be racalled and refinanced - this happens when the interest rates go down and the bond issuer pays up the face value and reissues the bonds at lower coupon rates, when this happens the original bond holders will have to reinvest in a less yielding instrument.

# Ranking our Fixed Income Dataset

We load our dataset which is comprised of treasuries, municipals (Munies), corporate and certificates of deposits (CDs). These can easily be retrieved from a broker's bond scanner:

In [ ]:
# Function to process data for each instrument type
def process_instrument_data(data_path, instrument_type):
    df = pd.read_csv(data_path)
    df['Instrument Type'] = instrument_type
    df['Ask'] = pd.to_numeric(df['Ask'], errors='coerce')
    df['Coupon'] = pd.to_numeric(df['Coupon'], errors='coerce')
    df.fillna(0, inplace=True)
    return df

def process_all_fixedinc_csvs():
    FILE_PREFIX_CAT = {
        'cds': 'Certificates of Deposit',
        'corp': 'Corporate Bonds',
        'munies': 'Municipal Bonds',
        'treas': 'Treasuries'
    }

    fixedinc_data = pd.DataFrame()
    for file_name in os.listdir(DATA_PATH):
        if file_name.endswith('.csv'):
            file_path = os.path.join(DATA_PATH, file_name)
            for prefix, instrument_type in FILE_PREFIX_CAT.items():
                if file_name.startswith(prefix):
                    df = process_instrument_data(file_path, instrument_type)
                    fixedinc_data = pd.concat([fixedinc_data, df], ignore_index=True)
    return fixedinc_data

fixedinc_data = process_all_fixedinc_csvs()
fixedinc_data.dropna(axis=1, how='all', inplace=True)
fixedinc_data.dropna(axis=0, inplace=True)

fixedinc_data = fixedinc_data[fixedinc_data['Ask'] > 0]
fixedinc_data['Ask Yield'] = pd.to_numeric(fixedinc_data['Ask Yield'], errors='coerce')
fixedinc_data = fixedinc_data[fixedinc_data['Ask Yield'] > 0]

fixedinc_data.to_csv(f'{DATA_PATH}/fixed_income.csv')
fixedinc_data.tail(2)

We set  constants for the market regime at the time of writing:

In [ ]:
TRADE_COSTS = 1 # Average $1 commmissions
TAX_RATE = 0.30 # The EU avarege for interest income
RF_RATE = (0.0375 * (1 - TAX_RATE)) # A savings account, interest rate taxed
INFLATION_RATE = 0.025
FACE_VALUE = 100.

DISCOUNT_RATE =  (1. + RF_RATE) / (1. + INFLATION_RATE) - 1.
DISCOUNT_RATE

We saw earlier that the YTM can be used as the discount rate, but this is stated in the secondary market - we might want to use our own rate.

Assume the risk free rate is a DGS insured european savings account average of 3.75% with interest taxed at 30%. We wont use the 10yr T-Bill as a proxy as we would have to factor in the secondary market yields and commissions. The RF rate is our opportunity cost of investing in a more risky bond, with inflation around 2.5%. 

With this info we construct our own discount rate, or the real rate with inflation and taxation.

## The Fisher equation for Real Rates

The Fisher formula calculates the real rates by compounding inflation:
$$
(1 + \text{Nominal Rate}) = (1 + \text{Real Rate})(1 + \text{Inflation Rate})
$$

Rearranging to solve for the real rate, you get:

$$
\text{Real Rate} = \frac{(1 + \text{Nominal Rate})}{(1 + \text{Inflation Rate})} - 1
$$

When interest rates are stated, these are not real or effective rates, to be effective, the nominal interest rate needs to be adjusted by tax and inflation (ATRR: After-Tax Real Returns).

## Stacking Bond Valuations

Using Fisher's equation to calculate our discount rate with inflation, and taxing coupon payments as below:

$$
DR =  \frac{1+(RF * (1-TR))}{(1+IR)} - 1
$$

Where:
- $RF$ is the risk free rate, at the time $4.75\%$
- $TR$ is the tax rate at $30\%$
- $IR$ is the inflation rate at $2.5\%$
- $DR$ is our adjusted discount rate.


The Bonds' PV can be calculated as follows:

$$
\text{Bond PV} = (\text{C} \times (\frac{1-\frac{1}{(1+DR)^N}}{DR}) + \frac{F}{(1 + DR)^N}) - \text{Commissions} - \text{Trade Costs}
$$

With this, we can rank the bonds by their PV:

In [ ]:
def calculate_bondvalue(coupon, ttm, ask_price, frequency, face_value=FACE_VALUE, discount = DISCOUNT_RATE, commission_rate=0., Trade_costs = TRADE_COSTS, tax_rate = TAX_RATE):
    if ask_price == 0:
        # Instrument did not have a feed.
        return 0.

    if coupon == 0 or frequency == 0 or frequency == "Principal at Maturity":
        frequency = 0  # For zero-coupon bonds or Tbills
        coupon = 0
    elif frequency == 'Annual':
        frequency = 1
    elif frequency == 'Semi-Annual':
        frequency = 2
    elif frequency == 'Quarterly':
        frequency = 4
    elif frequency == 'Monthly':
        frequency = 12
    else:
        raise ValueError(f"Unsupported frequency: {frequency}")

    if frequency == 0 or coupon == 0:
        bond_value = face_value / ((1 + discount) ** ttm)
    else:
        coupon_tax = coupon * tax_rate # interest is taxed as income
        coupon_effective = (coupon - coupon_tax) / frequency
        assert coupon_effective >= 0.

        # Need to return to the seller their share of coupon.
        total_coupons = int(np.ceil(ttm * frequency))

        accrued_interest = (coupon / frequency) * (total_coupons - ttm * frequency)
        coupon_dcf = (1 - (1 / (1 + discount) ** total_coupons)) / discount
        coupon_dcf = (face_value * coupon_effective) * coupon_dcf

        # Bonds gains at maturity are not taxed, only interest.
        face_dcf = face_value / (1 + discount) ** total_coupons
        bond_value = (coupon_dcf + face_dcf) - ((commission_rate * ask_price) + Trade_costs  + accrued_interest)

    return bond_value

fixedinc_data['PV'] = fixedinc_data.apply(lambda x: calculate_bondvalue(x['Coupon'] / 100.,
                                                              x['Time-To-Maturity (TTM)'],
                                                              x['Ask'],
                                                              x['Payment Frequency']), axis=1)


Given that the bonds have different attributes, we need to calculate an ROI and an Annualized rate, or ARR.

## Annnual Rate of Return (ARR)

Calculating the ARR of the fixed income instruments involes the following equation:

$$
ARR = \left( \frac{P + G}{P} \right)^{\frac{1}{n}} - 1
$$

Where:
- $P$ principal, or initial investment
- $G$ capital gains or losses
- $n$ number of years

With this, we can stack rank our bonds according to our requirements, and select the top 5 that beat the risk free rate, therefore it will be worth the effort to remove our cash from the savings account and buy the instrument from the secondary markets:

In [ ]:

fixedinc_data["ROI%"] = ((fixedinc_data["PV"] - fixedinc_data["Ask"]) / fixedinc_data["Ask"]) * 100
fixedinc_data["ARR%"] = (((fixedinc_data["PV"] / fixedinc_data["Ask"]) ** (1 / fixedinc_data['Time-To-Maturity (TTM)'])) - 1) * 100
fixedinc_data["ARR% VS RF%"] = fixedinc_data["ARR%"] - (RF_RATE * 100.)

df_ranked = fixedinc_data[["Financial Instrument", 'Instrument Type', "Time-To-Maturity (TTM)", "Coupon", 'Payment Frequency', "PV", "Ask", "Ask Yield", "ROI%", "ARR%", "ARR% VS RF%"]].sort_values(by="ARR% VS RF%", ascending=False)

df_ranked.head(5)

In [ ]:
df_ranked.tail(5)

Note that fixed income close to maturity can have an inflated ARR due to exponentiating small numbers. If we want to ladder bonds, we should look at longer TTMs:

In [ ]:
df_ranked = df_ranked[df_ranked["Time-To-Maturity (TTM)"] >= 1]
df_ranked.head(5)

In [ ]:
df_ranked.tail(5)

Looking at the distribution of the ARR vs RF, most good rating fixed income give a better yield than our savings account, with tax incentives (e.g. most capital gains at manturity are not taxed, expect the interest received unless its a Municipal Bond under certain conditions).

In [ ]:
df_ranked["ARR% VS RF%"].describe()

# Conclusion

In this article we explored the methodologies for bond valuation. By applying present value calculations and assessing the annual rate of return, we understand which fixed income security is fairly priced for us. 

Determining a bonds intrinsic values makes shaping and protecting our portfolios easier. ​

# References

- https://biz.libretexts.org/Bookshelves/Finance/Principles_of_Finance_(OpenStax)
- https://www.investopedia.com/ask/answers/accrued-interest-why-do-i-pay-when-i-buy-bond
- https://www.investopedia.com/terms/a/annual-return.asp
- https://www.investopedia.com/articles/investing/073113/introduction-treasury-securities.asp
- https://finance.ec.europa.eu/banking/banking-regulation/deposit-guarantee-schemes_en


## Github

Article and code available on [Github](https://github.com/adamd1985/Deep-Q-Learning-Applied-to-Algorithmic-Trading)

Kaggle notebook available [here](https://www.kaggle.com/code/addarm/deep-q-rl-with-algorithmic-trading-policy)

Google Collab available [here](https://colab.research.google.com/github/adamd1985/Deep-Q-Learning-Applied-to-Algorithmic-Trading/blob/main/drl_trading.ipynb)

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.